In [1]:
# import standard libraries
import datetime
import numpy as np
# import third-party libraries
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import norm
import wrds
# import local libraries

plt.close('all')

DOWNLOAD = True

In [4]:
# Create stock class

In [5]:
class Stock:
     
    def __init__(self, name, data):
        self.name = name
        self.data = data
        # compute winsorized data
        self.data_win = data[data.between(-0.04, 0.04, inclusive = False)]
        self.mean = None
        self.var = None
        self.mean_win = None
        self.var_win = None
        self.var95 = None
        self.var99 = None
        self.var95_norm = None
        self.var99_norm = None
        self.es95 = None
        self.es99 = None
        self.es95_norm = None
        self.es99_norm = None
        
    def computeMetrics(self):
        # compute mean and variance of normal data
        self.mean = self.data.mean()
        self.var = self.data.var()
        # compute mean and variance of winsorized data
        self.mean_win = self.data_win.mean()
        self.var_win = self.data_win.var()
        
    def printMetrics(self):
        print('\nMean of {} daily simple returns: {:.6f}'.format(self.name, self.mean))
        print('Variance of {} daily simple returns: {:.6f}'.format(self.name, self.var))
        print('Winsorized mean of {} daily simple returns: {:.6f}'.format(self.name, self.mean_win))
        print('Winsorized variance of {} daily simple returns: {:.6f}'.format(self.name, self.var_win))
        
    def plotDistribution(self):
        fig, ax = plt.subplots()
        self.data.plot.hist(bins = 100, density = True, ax = ax,
                            title = self.name,
                            label = 'Empirical density function',
                            color = '#6ab0e6',
                            )
        # plot normal distribution of data
        xmin, xmax = plt.xlim()
        x = np.linspace(xmin, xmax, 1000)
        p = norm.pdf(x, self.mean, np.sqrt(self.var))
        ax.plot(x, p, 'r', label = 'Normal density calibrated to data',
                color = 'r')
        # plot normal distribution of winsorized data
        p_win = norm.pdf(x, self.mean_win, np.sqrt(self.var_win))
        ax.plot(x, p_win, 'g', label = 'Normal density calibrated to winsorized data',
                color = 'g')
        
        plt.legend(prop={'size': 7})
    
    def computeQuantiles(self):
        # var
        self.var99 = self.data.quantile(0.01)
        self.var95 = self.data.quantile(0.05)
        self.var99_norm = norm.ppf(0.01, loc = self.mean, scale = np.sqrt(self.var))
        self.var95_norm = norm.ppf(0.05, loc = self.mean, scale = np.sqrt(self.var))
        
        # expected shortfall of empirical data
        sorted_data = self.data.sort_values()
        sorted_data_below_var99 = sorted_data[sorted_data <= self.var99]     
        sorted_data_below_var95 = sorted_data[sorted_data <= self.var95]
        self.es99 = sorted_data_below_var99.mean()
        self.es95 = sorted_data_below_var95.mean()
        
        # expected shortfall of normal distribution
        # self.es99_norm = (norm.cdf(self.var99, loc = self.mean, scale = np.sqrt(self.var)))
        # self.es95_norm = (norm.cdf(self.var95, loc = self.mean, scale = np.sqrt(self.var)))

        self.es99_norm = (norm.pdf(norm.ppf(0.01))*np.sqrt(self.var))/0.01 - self.mean
        self.es95_norm = (norm.pdf(norm.ppf(0.05))*np.sqrt(self.var))/0.05 - self.mean
        
    def printQuantiles(self):
        print('\n{}'.format(self.name))
        print('95% VaR (empirical): {:.6f}'.format(self.var95))
        print('99% VaR (empirical): {:.6f}'.format(self.var99))
        print('95% VaR (normal): {:.6f}'.format(self.var95_norm))
        print('99% VaR (normal): {:.6f}'.format(self.var99_norm))
        
        print('\n95% ES (empirical): {:.6f}'.format(self.es95))
        print('99% ES (empirical): {:.6f}'.format(self.es99))
        print('95% ES (normal): {:.6f}'.format(self.es95_norm))
        print('99% ES (normal): {:.6f}'.format(self.es99_norm))
                

In [ ]:
# run main

In [ ]:
if __name__ == '__main__':
    
    if DOWNLOAD:
        # connect to databse and download csv files (run once)
        db = wrds.Connection(wrds_username = 'wmartin')
        db.create_pgpass_file() # run once
    
        # download data
        aapl = db.raw_sql("select date, ret from crsp.dsf where permco in (7) and date>='1999-12-31' and date<='2019-12-31'")
        gs = db.raw_sql("select date, ret from crsp.dsf where permco in (35048) and date>='1999-12-31' and date<='2019-12-31'")
        msft = db.raw_sql("select date, ret from crsp.dsf where permco in (8048) and date>='1999-12-31' and date<='2019-12-31'")
        pg = db.raw_sql("select date, ret from crsp.dsf where permco in (21446) and date>='1999-12-31' and date<='2019-12-31'")
        ge = db.raw_sql("select date, ret from crsp.dsf where permco in (20792) and date>='1999-12-31' and date<='2019-12-31'")
    
        # save csv
        aapl.to_csv('aapl.csv')
        gs.to_csv('gs.csv')
        msft.to_csv('msft.csv')
        pg.to_csv('pg.csv')
        ge.to_csv('ge.csv')
    
    # read csvs
    use_cols = ['ret', 'date']
    index_col = 'date'
    
    aapl = pd.read_csv('aapl.csv', usecols = use_cols, index_col = index_col)
    gs = pd.read_csv('gs.csv', usecols = use_cols, index_col = index_col)
    msft = pd.read_csv('msft.csv', usecols = use_cols, index_col = index_col)
    pg = pd.read_csv('pg.csv', usecols = use_cols, index_col = index_col)
    ge = pd.read_csv('ge.csv', usecols = use_cols, index_col = index_col)

    # create stock objects
    stocks = dict(aapl = Stock('aapl', aapl['ret']),
                  gs = Stock('gs', gs['ret']),
                  msft = Stock('msft', msft['ret']),
                  pg = Stock('pg', pg['ret']),
                  ge = Stock('ge', ge['ret'])
                  )
    
    # (a) compute mean and variance of daily simple returns
    # (b) and winsorized mean and variance
    for _, stock in stocks.items():
        stock.computeMetrics()
        stock.printMetrics()
        
    # (b cont.)
    # plot distributions
    for _, stock in stocks.items():
        stock.plotDistribution()
        
    # (c) 
    for _, stock in stocks.items():
        stock.computeQuantiles()
        stock.printQuantiles()